In [3]:
#写一段代码实现一个fizz-buzz-fizzbuzz的小游戏
#从1开始往上数，3的倍数输出fizz，5的倍数输出buzz，15的倍数输出fizzbuzz，
#编码:就是将对应的输出规律变成计算机可识别的数字
def encode(i):
    if i%15==0:return 3
    elif i%5==0:return 2
    elif i%3==0:return 1
    else: return 0
#解码：就是将对应输出的编码找到针对真正对应的输出进行输出
def decode(i,prediction):
    return [str(i),'fizz','buzz','fizzbuzz'][prediction]
print(1,decode(1,encode(1)))
print(2,decode(2,encode(2)))
print(3,decode(3,encode(3)))
print(4,decode(4,encode(4)))
print(5,decode(5,encode(5)))
print(15,decode(15,encode(15)))

1 1
2 2
3 fizz
4 4
5 buzz
15 fizzbuzz


In [47]:
#搭建一个神经网络实现上述游戏
import torch
import numpy as np
NUM_DIGITS=10#将输入表示成10位的二进制格式，
def binary_encode(i,num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])
print(binary_encode(1000,10))#1111111000
#这里是有问题的吧，将数字127转化成二进制后，应该是0001111111，所以导致后面的训练是不对的，所以这个函数有问题！

[0 0 0 1 0 1 1 1 1 1]


In [45]:
trainX=torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(101,2**NUM_DIGITS)])
trainY=torch.LongTensor([encode(i) for i in range(101,2**NUM_DIGITS)])

In [25]:
print(trainX.shape)
print(trainY.shape)
#print(len(trainY))

torch.Size([923, 10])
torch.Size([923])


In [22]:
#搭建神经网络
NUM_H=100
model=torch.nn.Sequential(
          torch.nn.Linear(NUM_DIGITS,NUM_H),
          torch.nn.ReLU(),
          torch.nn.Linear(NUM_H,4))#最后就是4分类的输出

In [23]:
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.04)

In [28]:
#训练网络
BATCH_SIZE=128
for epoch in range(10000):
    for start in range(0,len(trainY),BATCH_SIZE):
#         x=trainX[start:start+BATCH_SIZE]
#         y_pred=model(x)
#         loss=loss_fn(y_pred,train_Y[start:start+BATCH_SIZE])
        end=start+BATCH_SIZE
        batchX=trainX[start:end]
        batchY=trainY[start:end]
        y_pred=model(batchX)
        loss=loss_fn(y_pred,batchY)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch%1000==0:
        print('epoch',epoch,'loss',loss.item())

epoch 0 loss 1.1678415536880493
epoch 1000 loss 0.5117358565330505
epoch 2000 loss 0.1281355619430542
epoch 3000 loss 0.04216296598315239
epoch 4000 loss 0.01959412358701229
epoch 5000 loss 0.011504154652357101
epoch 6000 loss 0.00762152997776866
epoch 7000 loss 0.005540934391319752
epoch 8000 loss 0.004279635846614838
epoch 9000 loss 0.003483657957985997


In [30]:
#利用训练好的模型，对1-100的数字进行预测。
testX=torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(1,101)])
with torch.no_grad():
    y_pred=model(testX)

In [39]:
#print(type(y_pred))
#print(y_pred.max(1))
#print(y_pred.max(1)[1].data.tolist())
predictions = zip(range(1,101),y_pred.max(1)[1].data.tolist())
print([decode(i,x) for (i,x) in predictions])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', '69', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', '81', '82', '83', '84', 'buzz', '86', '87', '88', '89', 'fizzbuzz', '91', '92', '93', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [43]:
print(np.sum(y_pred.max(1)[1].numpy()==np.array([encode(i) for i in range(1,101)])))

95


In [50]:
#the second time 
def encode(i):
    if i%15==0: return 3
    elif i%5==0: return 2
    elif i%3==0: return 1
    else: return 0
def decode(i,prediction):
    return [str(i),'fizz','buzz','fizzbuzz'][prediction]

In [51]:
print(1,decode(1,encode(1)))
print(3,decode(3,encode(3)))
print(5,decode(5,encode(5)))
print(15,decode(15,encode(15)))

1 1
3 fizz
5 buzz
15 fizzbuzz


In [52]:
#将十进制的数字编码成num_digits位的数字。
def binary_encode_(x,num_digits):
    return np.array([x>>d & 1 for d in range(num_digits)])
NUM_DIGITS=10
trainx=torch.Tensor([binary_encode(x,NUM_DIGITS) for x in range(101,2**NUM_DIGITS)])
trainy=torch.LongTensor([encode(i) for i in range(101,2**NUM_DIGITS)])
print(trainx.shape)
print(trainy.shape)

torch.Size([923, 10])
torch.Size([923])


In [55]:
NUM_DIGITS=10
NUM_H=100
model=torch.nn.Sequential(
       torch.nn.Linear(NUM_DIGITS,NUM_H),
       torch.nn.ReLU(),
       torch.nn.Linear(NUM_H,4))
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.04)

In [56]:
batch_size=128
for epoch in range(10000):
    for start in range(0,len(trainy),batch_size):
        end=start+batch_size
        batchx=trainx[start:end]
        batchy=trainy[start:end]
        y_pred=model(batchx)
        loss=loss_fn(y_pred,batchy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch%1000==0:
        print('epoch',epoch,'loss',loss.item())

epoch 0 loss 1.1490232944488525
epoch 1000 loss 0.38249239325523376
epoch 2000 loss 0.06652012467384338
epoch 3000 loss 0.028530113399028778
epoch 4000 loss 0.01735602878034115
epoch 5000 loss 0.010908828116953373
epoch 6000 loss 0.007348937448114157
epoch 7000 loss 0.0051686023361980915
epoch 8000 loss 0.0039175162091851234
epoch 9000 loss 0.0031071139965206385


In [63]:
testx=torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(1,101)])
with torch.no_grad():
    y_pred=model(testx)
predictions=zip(range(1,101),y_pred.max(1)[1].data.tolist())
print([decode(i,x) for (i,x) in predictions])
print(np.sum(y_pred.max(1)[1].numpy()==np.array([encode(i) for i in range(1,101)])))

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', '9', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', '21', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', '42', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', '81', '82', '83', 'fizz', 'buzz', '86', '87', '88', '89', 'fizzbuzz', '91', '92', '93', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']
94
